In [5]:
%pip install -r requirements.txt nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## NLP Pipeline + Probabilistic Language Models 
Building Unigram & Bigram Models from Real Text Data with NLTK

- Construct an end-to-end NLP pipeline including tokenization and normalization.
- Implement and analyze **Unigram** and **Bigram** language models.
- Apply **Maximum Likelihood Estimation (MLE)** to compute sentence probabilities.
- Explore how statistical models capture language patterns and dependencies.

## 1. Document Collection

In this section, we download and load text data from the Gutenberg Corpus using the nltk library. This corpus contains a selection of classic texts such as works by Shakespeare, Jane Austen, and others.

We begin by downloading the corpus.

Then, we extract the raw text from each file in the corpus.

Finally, we print the total number of documents available.

In [6]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

docs = [gutenberg.raw(file_id) for file_id in gutenberg.fileids()]
print(f"Total docs: {len(docs)}")
print(gutenberg.fileids())

Total docs: 18
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\shiru\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


## 2. Tokenizer Implementation

Tokenization is the process of breaking raw text into individual tokens (words). For this task, we use a regular expression-based tokenizer to:
- Convert text to lowercase.
- Remove punctuation and symbols.
- Split by word boundaries.

This simple tokenizer is fast and suitable for basic information retrieval tasks.

In [7]:
import re

def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# Apply tokenizer to all documents
tokenized_docs = [tokenize(doc) for doc in docs]
print("Sample tokens:", tokenized_docs[0][:20])

Sample tokens: ['emma', 'by', 'jane', 'austen', '1816', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', 'handsome', 'clever', 'and', 'rich', 'with', 'a', 'comfortable', 'home', 'and']


## 3. Normalization Pipeline

Normalization prepares tokens for indexing and comparison. It includes:
- Removing common stop words like "the", "is", "in".
- Applying stemming using the Porter Stemmer, which reduces words to their base form (e.g., "running" → "run").

This helps reduce vocabulary size and improve retrieval accuracy.

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def normalize(tokens):
    return [stemmer.stem(token) for token in tokens if token not in stop_words]

# Apply normalization
normalized_docs = [normalize(tokens) for tokens in tokenized_docs]
print("Normalized sample:", normalized_docs[0][:20])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Normalized sample: ['emma', 'jane', 'austen', '1816', 'volum', 'chapter', 'emma', 'woodhous', 'handsom', 'clever', 'rich', 'comfort', 'home', 'happi', 'disposit', 'seem', 'unit', 'best', 'bless', 'exist']


### Talking Points: Normalized Tokens Output ###
Reduced Vocabulary Size

Tokens are stemmed using PorterStemmer, so similar words (e.g., happiness, happy) are mapped to a common base (happi), reducing redundancy.

Stopwords Removed

Common English stopwords (e.g., was, and, the) have been successfully excluded, keeping only semantically rich words.

Lowercased and Tokenized

All tokens are in lowercase, ensuring consistency and avoiding duplication (e.g., Emma and emma treated the same).

Domain-Specific Terms Retained

Important character and domain-specific tokens like emma, jane, austen, and woodhous remain, which are useful for downstream tasks such as named entity recognition or topic modeling.

Examples of Stemming

Words like handsome → handsom, comfortable → comfort, happiness → happi demonstrate how morphological variants are reduced to their root form.

Text Now Ready for Modeling

The normalized output is a compact, semantically focused representation of the original text—ideal for building language models, calculating TF-IDF, or training classifiers.

## 4. Inverted Index Construction ###

An inverted index maps each term to the list of documents it appears in. This allows efficient document retrieval given a search query.

We build an inverted index using the normalized tokens from our document collection.

In [9]:
from collections import defaultdict

def build_inverted_index(docs):
    index = defaultdict(set)
    for doc_id, tokens in enumerate(docs):
        for token in tokens:
            index[token].add(doc_id)
    return {term: list(doc_ids) for term, doc_ids in index.items()}

inverted_index = build_inverted_index(normalized_docs)

# Example term
print("Docs containing 'scienc':", inverted_index.get('scienc'))

Docs containing 'scienc': [3, 8, 9, 10, 11, 12, 13, 17]


### Building an Inverted Index ###
In this section, we construct an inverted index, which is a data structure commonly used in information retrieval systems (like search engines).

What We Did:
We defined the build_inverted_index function to map each unique term to the set of document IDs where that term appears.

We applied this function to the normalized documents to build the index efficiently.

We then retrieved a sample result: all documents containing the term 'scienc' (note the stemmed form of science).

### Talking Points: Inverted Index Result ###
Efficient Retrieval

The inverted index enables fast lookup of which documents contain a given term, improving query efficiency in large corpora.

Supports Boolean Queries

This structure is foundational for building AND/OR/NOT query systems in search engines and recommender systems.

Stemming Normalization

The term 'scienc' is the stemmed form of science, meaning all morphological variants (e.g., scientific, sciences) map to this single key.

Document Coverage Insight

The output [3, 8, 9, 10, 11, 12, 13, 17] tells us which documents (by index) include the concept of science — useful for relevance ranking, search, or topic clustering.

Scalability

Using defaultdict(set) ensures that repeated terms across documents are aggregated efficiently, and conversion to a list ensures JSON-serializability.

## Part 2 – Probabilistic Language Models

###  Unigram Model
We build a unigram probabilistic language model for one document using Laplace (add-one) smoothing. This model estimates the probability of a query based on the frequency of individual tokens in the document.

Steps:
Token counts are computed using Counter.

Probabilities are calculated using:
                             p(w) = count(w) + 1/total tokens + vocab size
                            
 Log probabilities are used to avoid underflow in multiplication of small values.

In [10]:
from collections import Counter
import math

def unigram_model_with_laplace(doc_tokens, vocab):
    total_tokens = len(doc_tokens)
    vocab_size = len(vocab)
    token_counts = Counter(doc_tokens)

    def score(query_tokens):
        prob_log = 0.0
        for token in query_tokens:
            count = token_counts.get(token, 0)
            prob = (count + 1) / (total_tokens + vocab_size)
            prob_log += math.log(prob)
        return prob_log
    
    return score

In [11]:
# Choose one document and build its model
doc_id = 0
doc_tokens = normalized_docs[doc_id]
vocab = set(token for doc in normalized_docs for token in doc)

model = unigram_model_with_laplace(doc_tokens, vocab)

# Query scoring
query = "data science".split()
query_tokens = normalize(tokenize(" ".join(query)))

print(f"Query Score: {model(query_tokens)}")

Query Score: -23.01797544445517


**Talking Points: Unigram Model Output**

What the Score Means
The score -23.01 is the log-probability of the query "data science" appearing in the selected document (doc_id = 0). Lower (more negative) scores indicate lower likelihood.

Laplace Smoothing
Ensures that even unseen words get a small non-zero probability, avoiding issues like log(0).

Unigram Assumption
Assumes independence between query words, which simplifies the model but ignores word order and context.

Vocabulary-Aware Probability
Probability is adjusted based on both the number of tokens in the document and the entire vocabulary seen across the corpus.

Useful for Ranking Documents
This score can be used to rank multiple documents by how likely they are to generate a given query—an essential technique in information retrieval systems.

**Bigram Language Model (with Log-Probability Scoring)**

In this step, we train a bigram language model to capture word-to-word transitions. Unlike the unigram model, which assumes each word is independent, the bigram model considers the previous word to predict the current word.

What We Did:
We built bigram and unigram frequency tables:

bigram_counts[prev][curr]: Frequency of a word pair.

unigram_counts[prev]: Frequency of the first word in each bigram.

These counts will be used to calculate conditional probabilities:
P(w_i | w_{i-1}) = \frac{\text{count}(w_{i-1}, w_i)}{\text{count}(w_{i-1})}
$$  
Log-probabilities are used to avoid numerical underflow.



In [12]:
from collections import defaultdict, Counter

def train_bigram_model(tokens):
    bigram_counts = defaultdict(Counter)
    unigram_counts = Counter()

    for i in range(1, len(tokens)):
        prev_word = tokens[i - 1]
        curr_word = tokens[i]
        bigram_counts[prev_word][curr_word] += 1
        unigram_counts[prev_word] += 1

    return bigram_counts, unigram_counts

In [13]:
def bigram_probability(query_tokens, bigram_counts, unigram_counts, vocab_size):
    prob_log = 0.0

    for i in range(1, len(query_tokens)):
        prev = query_tokens[i - 1]
        curr = query_tokens[i]
        count_bigram = bigram_counts[prev].get(curr, 0)
        count_unigram = unigram_counts.get(prev, 0)

        # Laplace smoothing
        prob = (count_bigram + 1) / (count_unigram + vocab_size)
        prob_log += math.log(prob)

    return prob_log


In [14]:
# Train model on one document
doc_id = 0
doc_tokens = normalized_docs[doc_id]
vocab = set(token for doc in normalized_docs for token in doc)
vocab_size = len(vocab)

bigram_counts, unigram_counts = train_bigram_model(doc_tokens)

# Score a query
query = "data science is evolving"
query_tokens = normalize(tokenize(query))
query_score = bigram_probability(query_tokens, bigram_counts, unigram_counts, vocab_size)

print(f"Bigram log-probability score: {query_score}")

Bigram log-probability score: -20.337464559737946


**Talking point**

Contextual Modeling
The bigram model captures local word dependencies, making it more powerful than the unigram model for modeling natural language.

Interpretation of Score
The log-probability score of -20.34 indicates how likely a sequence (e.g., a query) is under the trained model. A less negative score means a higher likelihood.

Frequency-Based Estimation
The model relies purely on observed frequencies. If a bigram hasn’t been seen in training, it would assign it a zero probability—hence the need for smoothing (e.g., add-one smoothing) in practice.

Real-World Application
Bigram models are commonly used in text prediction, speech recognition, and autocomplete systems.

Next Step
Combine this model with smoothing techniques (like Laplace or Kneser-Ney) for better generalization, especially in sparse data scenarios.

In [15]:
# Sample tokens from Emma by Jane Austen
tokens = ['emma', 'by', 'jane', 'austen', '1816', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', 'handsome', 'clever', 'and', 'rich', 'with', 'a', 'comfortable', 'home', 'and']


### Laplace-smoothed bigram probability ###
We implemented a Laplace-smoothed bigram language model to estimate the probability of a sentence using frequency counts of unigrams and bigrams.

sentence = ['emma', 'was', 'clever']

The model uses the following formula for Laplace-smoothed bigram probability:

Where:

    P(wi|wi-1)=Count(wi-1,wi)+1/Count(wi-1)+V


V is the size of the vocabulary (number of unique words),

Laplace smoothing ensures no zero probabilities.

In [16]:

from collections import defaultdict
import math

# Step 1: Vocabulary
vocab = set(tokens)
V = len(vocab)

# Step 2: Count unigrams and bigrams
unigram_counts = defaultdict(int)
bigram_counts = defaultdict(int)

for i in range(len(tokens) - 1):
    unigram_counts[tokens[i]] += 1
    bigram = (tokens[i], tokens[i + 1])
    bigram_counts[bigram] += 1

# Add the last token to unigram count
unigram_counts[tokens[-1]] += 1

# Step 3: Define Laplace smoothed probability function
def laplace_bigram_prob(w1, w2):
    bigram = (w1, w2)
    return (bigram_counts[bigram] + 1) / (unigram_counts[w1] + V)

# Example sentence
sentence = ['emma', 'was', 'clever']
score = 1.0
for i in range(1, len(sentence)):
    prob = laplace_bigram_prob(sentence[i-1], sentence[i])
    score *= prob

print(f"Laplace-smoothed bigram probability for the sentence: {score:.10f}")


Laplace-smoothed bigram probability for the sentence: 0.0030959752


### Talking Points ###
Why Laplace Smoothing?

Adds 1 to each bigram count to avoid assigning zero probability to unseen word pairs.

Especially useful when the dataset is small or sparse.

What Does 0.0031 Mean?

This is the estimated probability of the full sentence "emma was clever" under the model.

It's a very low value, which is expected since multiplying probabilities < 1 quickly leads to small results.

Interpretation

A higher probability means the sentence is more natural or likely based on training data.

Even this small value can be comparatively high for short sequences, depending on corpus size and vocabulary.

Real-World Relevance

Language models like this one are foundational in:

Text prediction (e.g., keyboards)

Speech recognition

Machine translation

Chatbots and autocomplete systems

### Log-probability score ###
We implemented a bigram language model using Laplace smoothing (add-one smoothing) and computed the log-probability of a test sentence:
sentence = ['emma', 'was', 'clever']


### Key Steps ###
Tokenization: We converted the raw text from Jane Austen's Emma into lowercase tokens.

Bigram Count: We computed the frequency of all consecutive word pairs (bigrams).

Laplace Smoothing: We applied add-one smoothing to handle unseen bigrams and avoid zero probabilities.

Log Probability: log P(w1,w2,w3)=log P(w2|w1)+ log P(w3|w2)

Laplace-smoothed bigram log-probability: -5.7777

This means the natural log (ln) of the joint probability of the sentence "emma was clever" is -5.7777.

In [17]:
# Step 4: Compute log-probability score 
log_score = 0.0
for i in range(1, len(sentence)):
    prob = laplace_bigram_prob(sentence[i-1], sentence[i])
    log_score += math.log(prob)

print(f"Laplace-smoothed bigram log-probability: {log_score:.4f}")


Laplace-smoothed bigram log-probability: -5.7777


### Talking Points ####
Why Log-Probability?

Probabilities get very small when multiplying many bigrams, so using log-probability prevents underflow.

Log-space makes it easier to compare sentence likelihoods.

Why Laplace Smoothing?

Helps avoid zero probability for unseen word pairs.

Ensures all bigrams, even unseen ones, have a non-zero value.

How to Interpret -5.7777?

This is a reasonably high probability for a 3-word sentence.

A more negative value means lower likelihood under the model.

You can convert this back to actual probability:
    P=e −5.7777 ≈0.0031
Model Use Cases

Can rank sentences based on likelihood.

Forms the foundation for spelling correction, text generation, and speech recognition.



### Summary: Bigram Language Model

The Bigram Model estimates the probability of a word given the one before it, allowing us to model word dependencies and improve accuracy over the Unigram Model. To handle sparsity and zero probabilities, we use Laplace smoothing.

This model brings us closer to real-world text generation and ranking but can still struggle with long-range dependencies, which more advanced models like trigrams or neural LMs aim to solve.
